**Notebook imports**

In [ ]:
import zipfile
import numpy as np
import pandas as pd
import os

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving SpamData.zip to SpamData.zip


In [ ]:
zip_file = 'SpamData.zip'
unzip_dir = '/content/DataExtracted'

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

**Constants**

In [ ]:
TRAINING_DATA_FILE = 'DataExtracted/SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'DataExtracted/SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'DataExtracted/SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'DataExtracted/SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'DataExtracted/SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'DataExtracted/SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'DataExtracted/SpamData/03_Testing/test-target.txt'
VOCAB_SIZE = 2500

**Read and Load Features from .txt files into NumPy Array**

In [ ]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [ ]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [ ]:
sparse_train_data[:5]

array([[ 0,  0,  1,  3],
       [ 0,  8,  1,  1],
       [ 0, 17,  1,  1],
       [ 0, 19,  1,  1],
       [ 0, 20,  1,  1]])

In [ ]:
sparse_train_data[-5:]

array([[5795, 1350,    0,    1],
       [5795, 1959,    0,    1],
       [5795, 2288,    0,    1],
       [5795, 2366,    0,    2],
       [5795, 2377,    0,    1]])

In [ ]:
print('Number of rows in training file: ', sparse_train_data.shape[0])
print('Number of rows in test file: ', sparse_test_data.shape[0])

Number of rows in training file:  262923
Number of rows in test file:  113238


In [ ]:
print('Number of emails in tarining file:', np.unique(sparse_train_data[:, 0]).size)

Number of emails in tarining file: 4013


In [ ]:
print('Number of emails in test file:', np.unique(sparse_test_data[:, 0]).size)

Number of emails in test file: 1724


**How to create Empty DataFrame**

In [ ]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [ ]:
len(column_names)

2502

In [ ]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [ ]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names )
full_train_data.fillna(value=0, inplace=True)

In [ ]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Create a Full Matrix from a Sparse Matrix**

In [ ]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
  """
  Form a full matrix from a sparse matrix. Return a pandas dataframe.
  Keyword arguments:
  sparse_matrix -- numpy array
  nr_words -- size of the vocabulary . Total number of tokens.
  doc_idx -- position of the document id in the sparse matrix. Default: 1st column
  word_idx -- position of the word id in the sparse matrix. Default: 2nd column
  cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd
  freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
  """
  column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
  doc_id_names = np.unique(sparse_matrix[:, 0])
  full_matrix = pd.DataFrame(index=index_names, columns=column_names )
  full_matrix.fillna(value=0, inplace=True)

  for i in range(sparse_matrix.shape[0]):
    doc_nr = sparse_matrix[i][doc_idx]
    word_id = sparse_matrix[i][word_idx]
    label = sparse_matrix[i][cat_idx]
    occurrence = sparse_matrix[i][freq_idx]

    full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
    full_matrix.at[doc_nr, 'CATEGORY'] = label
    full_matrix.at[doc_nr, word_id] = occurrence

  full_matrix.set_index('DOC_ID', inplace=True)
  return full_matrix


In [ ]:
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

In [ ]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,3,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,5,1,0,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,2,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,1,3,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


**Training the Naive Bayes Model**

Calculating the Probability of Spam

In [ ]:
full_train_data.CATEGORY.size

4013

In [ ]:
full_train_data.CATEGORY.sum()

1248

In [ ]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Probability of spam is: ', prob_spam)

Probability of spam is:  0.310989284824321


**Total Number of Words/Tokens**

In [ ]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY' ]
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,3,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,1,0,1,2,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,2,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,1,3,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4013,)

In [ ]:
email_lengths[:5]

,0
DOC_ID,
0,66
1,10
2,238
3,183
4,39


In [ ]:
total_wc = email_lengths.sum()
total_wc

436622

**Number of Tokens in Spam & Ham Emails**

In [ ]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape

(1248,)

In [ ]:
spam_wc = spam_lengths.sum()
spam_wc

186616

In [ ]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape

(2765,)

In [ ]:
email_lengths.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [ ]:
nonspam_wc = ham_lengths.sum()
nonspam_wc

250006

In [ ]:
spam_wc + nonspam_wc - total_wc

0

In [ ]:
print('Average nr of words in spam emails {:.0f}'.format(spam_wc / spam_lengths.shape[0]))
print('Average nr of words in ham emails {:.3f}'.format(nonspam_wc / ham_lengths.shape[0]))

Average nr of words in spam emails 150
Average nr of words in ham emails 90.418


**Summing the Tokens  Occuring in Spam**

In [ ]:
full_train_features.shape

(4013, 2500)

In [ ]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,3,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,1,0,1,2,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,2,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,1,3,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1885,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,0,0,2,4,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,3,0,0,0,1,0,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
1890,0,0,0,0,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1895,0,0,1,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_spam_tokens.shape

(1248, 2500)

In [ ]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1
summed_spam_tokens.shape

(2500,)

In [ ]:
summed_spam_tokens.tail()

,0
2495,23
2496,26
2497,1
2498,8
2499,13


**Summing the Tokens Occuring in Ham**

In [ ]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1

In [ ]:
summed_ham_tokens.shape

(2500,)

In [ ]:
summed_ham_tokens.tail()

,0
2495,1
2496,3
2497,26
2498,21
2499,20


In [ ]:
train_ham_tokens[2499].sum() + 1

20

**P(Token| Spam) - Probability that a Token Occurs given the Email is Spam**

In [ ]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

,0
0,0.010681
1,0.004881
2,0.007350
3,0.011406
4,0.006879


In [ ]:
prob_tokens_spam.sum()

1.0

**P(Token| Ham) - Probability that a Token Occurs given the Email is Spam**

In [ ]:
prob_tokens_nonspam = summed_ham_tokens/ (nonspam_wc + VOCAB_SIZE)
prob_tokens_nonspam.sum()

0.9999999999999999

**P(Token) - Probability that Token Occurs**

In [ ]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc

In [ ]:
prob_tokens_all.sum()

1.0

**Save the Trained MOdel**

In [ ]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

**Prepare Test Data**

In [ ]:
sparse_test_data.shape

(113238, 4)

In [ ]:
full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

In [ ]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [ ]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)